In [2]:
import pandas as pd 
import numpy as np
import tensorflow as tf
from tensorflow import keras 
import tensorflow.keras.preprocessing.text as tfkpt
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
import csv
import json

In [2]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


In [3]:
wine_df = pd.read_csv('wine_data/tipsy_sentiments/ranked_wine.csv')
wine_df = wine_df.dropna()
wine_df = wine_df[['description', 'sentiment']]
wine_df

,description,sentiment
0,This wine has very little going on aromaticall...,0
1,This Merlot has not fully ripened; with aromas...,0
2,Picture grandma standing over a pot of stewed ...,0
3,A white this age should be fresh and crisp; th...,0
4,This wine is a medium cherry-red color; with s...,0
...,...,...
129961,A hugely powerful wine; full of dark; brooding...,2
129962,This latest incarnation of the famous brand is...,2
129963,This new release from a great vintage for Char...,2
129964,This wine dazzles with perfection. Sourced fro...,2


In [4]:
wine_x = wine_df.description

In [5]:
tokenizer = Tokenizer() 
tokenizer.fit_on_texts(wine_x)
tokenizer.get_config()

640, "showchases": 36641, "kit": 36642, "singh": 36643, "tweaked": 36644, "hewitt": 36645, "cabernet\\u2014tight": 36646, "valley\\u2014soft": 36647, "herbal\\u2014but": 36648, "kahn": 36649, "greats": 36650, "willow\'s": 36651, "potential\\u2014drink": 36652, "mainstays": 36653, "uplands": 36654, "fruit\\u2014showing": 36655, "best\\u2014firm": 36656, "554": 36657, "babe": 36658, "tinting": 36659, "prominently\\u2014it": 36660, "metaphor": 36661, "gloved": 36662, "vermilion": 36663, "addiction": 36664, "sinous": 36665, "tentacles": 36666, "run\\u2014but": 36667, "pioneer\'s": 36668, "herbs\\u2014from": 36669, "dill\\u2014alongside": 36670, "now\\u20132029": 36671, "figuero": 36672, "style\\u2014aided": 36673, "dosage\\u2014rounded": 36674, "guerneville": 36675, "boeschen": 36676, "skiff": 36677, "seizure": 36678, "lattin": 36679, "patisserie": 36680, "wilful": 36681, "diversit\\u00e9": 36682, "echanges": 36683, "bourgignon": 36684, "oxymoron": 36685, "2013\\u20132016": 36686, "tranche

In [6]:
wine_x = tokenizer.texts_to_sequences(wine_x)


In [7]:
wine_x = tokenizer.sequences_to_matrix(wine_x)


In [8]:
wine_y = wine_df.sentiment
print(type(wine_y))

In [9]:
wine_y = keras.utils.to_categorical(wine_y, 3)

In [10]:
model = Sequential()
#Layer 1: activation - Sigmoid, dropout 1 at 10%
model.add(Dense(677, activation='sigmoid'))
model.add(Dropout(0.1))
#Layer 2: activation - Rectified Linear Unit, dropout 1 at 20%
model.add(Dense(557, activation='relu'))
model.add(Dropout(0.2))
#Layer 3: activation - Linear, dropout 1 at 30%
model.add(Dense(419, 'linear'))
model.add(Dropout(0.3))
#Layer 4: activation - Exponential Linear Unit, dropout 1 at 40%
model.add(Dense(281, activation='elu'))
model.add(Dropout(0.4))
#Layer 5: activation - Softmax, no dropout
model.add(Dense(3, activation='softmax'))

In [11]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [12]:
model.fit(wine_x, wine_y, 
batch_size=37, 
epochs=7, 
verbose=1, 
validation_split=0.2)

Train on 103972 samples, validate on 25994 samples
Epoch 1/7
103972/103972 [==============================] - 461s 4ms/sample - loss: 0.4901 - accuracy: 0.7603 - val_loss: 2.0766 - val_accuracy: 0.8684
Epoch 2/7
103972/103972 [==============================] - 465s 4ms/sample - loss: 0.4076 - accuracy: 0.8084 - val_loss: 3.3263 - val_accuracy: 0.8838
Epoch 3/7
103972/103972 [==============================] - 449s 4ms/sample - loss: 0.3789 - accuracy: 0.8217 - val_loss: 2.9519 - val_accuracy: 0.8675
Epoch 4/7
103972/103972 [==============================] - 449s 4ms/sample - loss: 0.3534 - accuracy: 0.8298 - val_loss: 4.0887 - val_accuracy: 0.8659
Epoch 5/7
103972/103972 [==============================] - 448s 4ms/sample - loss: 0.3316 - accuracy: 0.8376 - val_loss: 4.5498 - val_accuracy: 0.8776
Epoch 6/7
103972/103972 [==============================] - 4598s 44ms/sample - loss: 0.3079 - accuracy: 0.8474 - val_loss: 6.1390 - val_accuracy: 0.8721
Epoch 7/7
103972/103972 [================

In [15]:
wine_model = model.to_json()
with open('hot_wine_model.json', 'w') as json_file: 
  json_file.write(wine_model)
model.save_weights('hot_wine_model.h5') 